In [ ]:
%matplotlib widget
from spectraclass.data.base import DataManager
from typing import List, Union, Tuple, Optional, Dict, Callable
from spectraclass.data.spatial.tile.manager import TileManager, tm
import ipywidgets as ipw
import time, xarray as xa

t0 = time.time()
dm: DataManager = DataManager.initialize( "mgr.1", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
dm.modal.ext = "_img"
dm.proc_type = "cpu"
TileManager.block_size = 250
nimages = len( dm.modal.image_names )
print( f"found {nimages} images "  )
print( f"Completed init in {time.time()-t0} sec")

In [ ]:
viewer = None
file_selector = dm.modal.file_selector

def on_image_change( event: Dict ):
    dm.modal.set_current_image( file_selector.index )
    data: xa.DataArray = tm().tile.data
    print( f"Loading Image[{tm().image_index}]: {tm().image_name}" )
    print( f"Data shape = {data.shape}, coords = {data.coords.keys()}"  )

dm.modal.set_file_selection_observer( on_image_change )        
ipw.HBox( [ viewer, file_selector ], layout=ipw.Layout(flex='1 1 auto') )